## Introduction



We're thinking about the problem of finding the cheapest possible
nutritious diet. As we have discussed, this can be set up as a linear program.

$$
    \min_x p'x
$$

such that
$$
Ax \geq b
$$

where $p$ is a vector of prices, $A$ is a matrix that maps
vectors of quantities of food into vectors of nutrients, and where b is a vector of nutrient constraints.

The goal of this notebook is to expose the class to some data sources and to generate a minimum cost diet for the United States using recent national average price data.



## Food and Nutrient Data



The USDA maintains a database of foods called the [FNDDS](https://www.ars.usda.gov/northeast-area/beltsville-md-bhnrc/beltsville-human-nutrition-research-center/food-surveys-research-group/docs/fndds/) (Food and Nutrition Database for Dietary Studies). In it, they define "recipes," which are made up of "ingredients." There are one or more ingredients per recipe. Ingredients are then mapped to nutrients in terms of nutrient units per 100 g of an ingredient. Thus, using these mappings, we can generate a FNDDS recipe-level food-to-nutrient matrix we can use as our "A" matrix.

Lets first define a helper function we will use to make sure ids are formatted similarly (to help with dataframe merging). 



In [1]:
def format_id(id,zeropadding=0):
    """Nice string format for any id, string or numeric.

    Optional zeropadding parameter takes an integer
    formats as {id:0z} where
    """
    if pd.isnull(id) or id in ['','.']: return None

    try:  # If numeric, return as string int
        return ('%d' % id).zfill(zeropadding)
    except TypeError:  # Not numeric
        return id.split('.')[0].strip().zfill(zeropadding)
    except ValueError:
        return None

Lets load in the recipes and see what we have.



In [2]:
import pandas as pd
recipes = (pd.read_csv("Data/fndds_recipes.csv", )
           .assign(parent_foodcode = lambda df: df["parent_foodcode"].apply(format_id),
                   ingred_code = lambda df: df["ingred_code"].apply(format_id))
           .rename(columns={"parent_desc": "recipe"}))


# lets see an example of a recipe.. note 
recipes[recipes["recipe"].str.contains("Vegetable lasagna", case=False)]

,parent_foodcode,recipe,ingred_code,ingred_desc,ingred_wt
30351,58301110,"Vegetable lasagna, frozen meal",1029,"Cheese, mozzarella, low moisture, part-skim",8.0
30352,58301110,"Vegetable lasagna, frozen meal",1032,"Cheese, parmesan, grated",20.0
30353,58301110,"Vegetable lasagna, frozen meal",1036,"Cheese, ricotta, whole milk",12.0
30354,58301110,"Vegetable lasagna, frozen meal",1050,"Cream, fluid, light (coffee cream or table cream)",30.0
30355,58301110,"Vegetable lasagna, frozen meal",1151,"Milk, nonfat, fluid, without added vitamin A a...",20.0
30356,58301110,"Vegetable lasagna, frozen meal",2047,"Salt, table, iodized",2.2
30357,58301110,"Vegetable lasagna, frozen meal",4322,"Vegetable oil, averaged",8.0
30358,58301110,"Vegetable lasagna, frozen meal",4582,"Oil, canola",3.0
30359,58301110,"Vegetable lasagna, frozen meal",4610,"Margarine, regular, 80% fat, composite, stick,...",11.0
30360,58301110,"Vegetable lasagna, frozen meal",11090,"Broccoli, raw",100.0


And we can take a look at the ingredients. I'll call this dataframe "nutrition" because that's what we really care about in here.



In [3]:
nutrition = (pd.read_csv("Data/fndds_ingredients_nutrients.csv")
             .assign(ingred_code = lambda df: df["ingred_code"].apply(format_id)))

display(nutrition.head())
nutrition.columns

,ingred_code,Ingredient description,Capric acid,Lauric acid,Myristic acid,Palmitic acid,Palmitoleic acid,Stearic acid,Oleic acid,Linoleic Acid,...,Vitamin B12,"Vitamin B-12, added",Vitamin B6,Vitamin C,Vitamin D,Vitamin E,"Vitamin E, added",Vitamin K,Water,Zinc
0,1001,"Butter, salted",2.529,2.587,7.436,21.697,0.961,9.999,19.961,2.728,...,0.17,0.0,0.003,0.0,0.0,2.32,0.0,7.0,15.87,0.09
1,1002,"Butter, whipped, with salt",2.039,2.354,7.515,20.531,1.417,7.649,17.370,2.713,...,0.07,0.0,0.008,0.0,0.0,1.37,0.0,4.6,16.72,0.05
2,1003,"Butter oil, anhydrous",2.495,2.793,10.005,26.166,2.228,12.056,25.026,2.247,...,0.01,0.0,0.001,0.0,0.0,2.80,0.0,8.6,0.24,0.01
3,1004,"Cheese, blue",0.601,0.491,3.301,9.153,0.816,3.235,6.622,0.536,...,1.22,0.0,0.166,0.0,0.5,0.25,0.0,2.4,42.41,2.66
4,1005,"Cheese, brick",0.585,0.482,3.227,8.655,0.817,3.455,7.401,0.491,...,1.26,0.0,0.065,0.0,0.5,0.26,0.0,2.5,41.11,2.60


Index(['ingred_code', 'Ingredient description', 'Capric acid', 'Lauric acid',
       'Myristic acid', 'Palmitic acid', 'Palmitoleic acid', 'Stearic acid',
       'Oleic acid', 'Linoleic Acid', 'Linolenic Acid', 'Stearidonic acid',
       'Eicosenoic acid', 'Arachidonic acid', 'Eicosapentaenoic acid',
       'Erucic acid', 'Docosapentaenoic acid', 'Docosahexaenoic acid',
       'Butyric acid', 'Caproic acid', 'Caprylic acid', 'Alcohol', 'Caffeine',
       'Calcium', 'Carbohydrate', 'Carotene, alpha', 'Carotene, beta',
       'Cholesterol', 'Choline', 'Copper', 'Cryptoxanthin, beta', 'Energy',
       'Fatty acids, total monounsaturated',
       'Fatty acids, total polyunsaturated', 'Fatty acids, total saturated',
       'Dietary Fiber', 'Folate, DFE', 'Folate, food', 'Folate', 'Folic acid',
       'Iron', 'Lutein + zeaxanthin', 'Lycopene', 'Magnesium', 'Niacin',
       'Phosphorus', 'Potassium', 'Protein', 'Retinol', 'Riboflavin',
       'Selenium', 'Sodium', 'Sugars, total', 'Theobromin

Lets merge these mappings to end up with a matrix of recipes and their nutrients as measured by units of a nutrient per 100 grams of that food.



In [4]:
# normalize weights to percentage terms. 
recipes['ingred_wt'] = recipes['ingred_wt']/recipes.groupby(['parent_foodcode'])['ingred_wt'].transform("sum")

# we're going to extend the recipes data frame to include the nutrient profiles of its ingredients (in 100g)
df = recipes.merge(nutrition, how="left", on="ingred_code")

# multiply all nutrients per 100g of an ingredient by the weight of that ingredient in a recipe.
numeric_cols = list(df.select_dtypes(include=["number"]).columns)
numeric_cols.remove("ingred_wt")
df[numeric_cols] = df[numeric_cols].mul(df["ingred_wt"], axis=0)

# sum nutrients of food codes (over the multiple ingredients)
# python tip: one can merge dictionaries dict1 dict2 using **, that is: dict_merge = {**dict1, **dict2}. The ** effectively "unpacks" the key value pairs in each dictionary
df = df.groupby('parent_foodcode').agg({**{col: "sum" for col in numeric_cols},
                                        "recipe": "first"})

df.index.name = "recipe_id"

food_names = df["recipe"]

If we recall, the $ A  $ matrix maps foods into constrained nutrients. This is effectively the transpose of our `df`. We'll have to do more before we create `A` - not all of these nutrients are constrained, and unfortunately, we won't have prices for all foods.



## Prices



Now that we have the bones of the matrix $ A $, lets next consider the price vector $ p $. The USDA generates national average prices ([Purchase to Plate](https://www.ers.usda.gov/data-products/purchase-to-plate)) for these FNDDS foods using scanner data from grocery stores all over the country. These are in USD per 100 grams of a recipe. They have been doing this for a while, and they produce the prices in two-year batches.



In [5]:
prices = pd.read_csv("Data/fndds_national_prices.csv", usecols=['year', 'food_code','price_100gm'])
prices["food_code"] = prices["food_code"].apply(format_id)

prices = prices.set_index(["year", "food_code"])
print(prices.index.levels[0])

# we'll focus on the latest price data
prices = prices.xs("2017/2018", level="year")

# drop rows of prices where the price is "NA"
prices = prices.dropna(subset='price_100gm')

print(f"We have prices for {prices.shape[0]} unique recipes (FNDDS food codes)")

Index(['2011/2012', '2013/2014', '2015/2016', '2017/2018'], dtype='object', name='year')
We have prices for 4435 unique recipes (FNDDS food codes)


## Dietary Requirements



As before, we'll get our dietary requirements from the USDA.



In [6]:
rda = pd.read_csv("Data/rda.csv")
rda = rda.set_index("Nutrient")

rda.columns

Index(['Nutrient Type', 'Unit', 'Constraint Type', 'Child_1_3', 'Female_4_8',
       'Male_4_8', 'Female_9_13', 'Male_9_13', 'Female_14_18', 'Male_14_18',
       'Female_19_30', 'Male_19_30', 'Female_31_50', 'Male_31_50',
       'Female_51U', 'Male_51U'],
      dtype='object')

## Putting It All Together



Earlier, we generated a dataframe of foods and nutrients. This included something like 65 different nutrients over 8,900 recipes! Unfortunately, our price data far fewer foods, so we have to narrow the set of foods from which we are choosing. I'll solve this issue by taking the set intersection of the two sets of food codes, and then select those common food codes from both dataframes.



In [7]:
common_recipes = df.index.intersection(prices.index)

# python tip: given a list of indices, "loc" both subsets and sorts. 
df = df.loc[common_recipes]
prices = prices.loc[common_recipes]

# lets remap the price dataframe index to be the actual food names.
prices.index = prices.index.map(food_names)

A_all = df.T

A<sub>all</sub> will have the same number of foods as p has prices, but we now must to trim down the number of nutrients to include only those for which we have constraints. We'll look at the shapes of all these objects to be sure that the matrix multiplication operations are well defined.



In [8]:
# pick a demographic (column from rda dataframe)
'''
select from 
['Child_1_3', 'Female_4_8', 'Male_4_8', 'Female_9_13', 'Male_9_13', 
'Female_14_18', 'Male_14_18','Female_19_30', 'Male_19_30', 
'Female_31_50', 'Male_31_50', 'Female_51U', 'Male_51U']
'''
group = "Female_19_30"

# create lower bounds and upper bounds.
bmin = rda.loc[rda['Constraint Type'].isin(['RDA', 'AI']), group]
bmax = rda.loc[rda['Constraint Type'].isin(['UL']), group]

# reindex ensures we only keep nutrients in bmin/bmax
Amin = A_all.reindex(bmin.index).dropna(how='all')
Amax = A_all.reindex(bmax.index).dropna(how='all')

b = pd.concat([bmin, -bmax])
A = pd.concat([Amin, -Amax])

#python tip: by typing "=" after the variable name inside the curly braces, it formats the output so we don't have to write f"variable = {variable}"
print(f"{bmin.shape=}")
print(f"{Amin.shape=}")
print(f"{bmax.shape=}")
print(f"{Amax.shape=}")
print(f"{b.shape=}")
print(f"{A.shape=}")
print(f"{prices.shape=}")

bmin.shape=(26,)
Amin.shape=(26, 4426)
bmax.shape=(1,)
Amax.shape=(1, 4426)
b.shape=(27,)
A.shape=(27, 4426)
prices.shape=(4426, 1)


## Solving the Problem



First, we find a solution to the problem



In [9]:
from  scipy.optimize import linprog as lp
import numpy as np
p = prices
tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros
result = lp(p, -A, -b, method='highs')

result

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 2.4413443120658607
              x: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
            nit: 20
          lower:  residual: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
                 marginals: [ 1.090e-02  1.071e-02 ...  8.545e-02
                              4.730e-01]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  3.071e+01 ...  2.539e+02
                              2.510e+02]
                 marginals: [-4.930e-05 -0.000e+00 ... -0.000e+00
                             -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip

Let's interpret this.  Start with the cost of the solution:



In [10]:
print(f"Cost of diet for {group} is ${result.fun:.2f} per day.")

Cost of diet for Female_19_30 is $2.44 per day.


Next, what is it we're actually eating?



In [11]:
# lets mess with the index on price df so they are recipe names not ids.

# get the result x in a series with food names
diet = pd.Series(result.x,index=prices.index)

print("\nYou'll be eating (in 100s of grams or milliliters):")
print(round(diet[diet >= tol], 2))


You'll be eating (in 100s of grams or milliliters):
Milk, low fat (1%)                                   8.41
Carp, steamed or poached                             0.06
Egg, yolk only, raw                                  0.09
Split peas, from dried, fat added                    4.84
Rice, white, cooked, made with margarine             1.08
Cereal (Kellogg's All-Bran Complete Wheat Flakes)    0.05
Cereal, toasted oat                                  0.19
Ripe plantain, raw                                   3.23
dtype: float64


Given this diet, what are nutritional outcomes?



In [12]:
tab = pd.DataFrame({"Outcome":A.to_numpy()@diet.to_numpy(),"Recommendation":np.abs(b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)


With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               2000.0          2000.0
Protein           76.714117            46.0
Carbohydrate     276.775494           130.0
Dietary Fiber     47.781799            28.0
Linoleic Acid      21.96157            12.0
Linolenic Acid     2.751783             1.1
Calcium         1219.941986          1000.0
Iron                   18.0            18.0
Magnesium        417.695502           310.0
Phosphorus      1568.466207           700.0
Potassium            4700.0          4700.0
Zinc              13.983042             8.0
Copper              1.31819             0.9
Selenium               55.0            55.0
Vitamin A       1277.799363           700.0
Vitamin E              15.0            15.0
Vitamin D              15.0            15.0
Vitamin C              75.0            75.0
Thiamin             1.81167             1.1
Riboflavin         2.6

Finally, what are the constraints that bind?  Finding a less expensive
diet might involve finding less expensive sources for these particular nutrients.



In [13]:
print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol].index.tolist())


Constraining nutrients are:
['Energy', 'Iron', 'Potassium', 'Selenium', 'Vitamin E', 'Vitamin D', 'Vitamin C', 'Choline']
